In [2]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam
from VariationalBayes.NormalParams import MVNParam, UVNParam
from VariationalBayes.GammaParams import GammaParam

from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np

import VariationalBayes as vb


def ApproxEq(v1, v2, tol=1e-8):
    return np.max(np.abs(v1 - v2)) < tol

In [3]:
vec = np.array([0.5, 2.5])
for lb in [ -float("inf"), 0.2]:
    for ub in [ float("inf"), 2.7]:
        vec_free = vb.Parameters.unconstrain_vector(vec, lb=lb, ub=ub)
        vec_test = vb.Parameters.constrain(vec_free, lb=lb, ub=ub)
        assert ApproxEq(vec, vec_test)

In [4]:
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)
vec = vb.Parameters.VectorizeSymMatrix(mat)
assert ApproxEq(mat, vb.Parameters.UnvectorizeSymMatrix(vec))


In [11]:
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)
vec = vb.Parameters.VectorizeSymMatrix(mat)

UnvecJac = jacobian(vb.Parameters.UnvectorizeSymMatrix)
print UnvecJac(vec)

VecJac = jacobian(vb.Parameters.VectorizeSymMatrix)
print VecJac(mat)


[[[ 1.  0.  0.]
  [ 0.  1.  0.]]

 [[ 0.  1.  0.]
  [ 0.  0.  1.]]]
[[[ 1.  0.]
  [ 0.  0.]]

 [[ 0.  1.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  1.]]]


In [5]:
# works
def MyFun(vec):
    return np.concatenate((vec[0:2], vec[2:4])).reshape(2,2)

vec = np.array([1., 2., 3., 4.])
MyFun(vec)

MyFunJacobian = jacobian(MyFun)
MyFunJacobian(vec)

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]])

In [7]:
# Does not work
def MyFun(vec):
    foo = vec[0:2]
    foo = np.append(foo, vec[2:4])
    return foo

vec = np.array([1., 2., 3., 4.])
MyFun(vec)

MyFunJacobian = jacobian(MyFun)
# MyFunJacobian(vec)

NotImplementedError: Gradient of append not yet implemented.

In [ ]:
# Do it my damn self

from autograd.core import primitive

# Uses 0-indexing. (row, col) = (k1, k2)
def SymIndex(k1, k2):
    def LDInd(k1, k2):
        return k2 + k1 * (k1 + 1) / 2

    if k2 <= k1:
        return LDInd(k1, k2)
    else:
        return LDInd(k2, k1)
    
@primitive
def UnvectorizeMatrix(vec):
    mat_size = int(0.5 * (math.sqrt(1 + 8 * vec.size) - 1))
    if mat_size * (mat_size + 1) / 2 != vec.size: \
        raise ValueError('Vector is an impossible size')
    mat = np.empty((mat_size, mat_size))
    for k1 in range(mat_size):
        for k2 in range(mat_size):
            mat[k1, k2] = vec[SymIndex(k1, k2)]
    return mat


def UnvectorizeMatrix_vjp(g, ans, vs, gvs, vec):
    assert g.shape[0] == g.shape[1]
    mat_size = g.shape[0]
    vjp = np.zeros(mat_size * (mat_size + 1) / 2)
    for k1 in range(mat_size):
        for k2 in range(mat_size):
            vjp[SymIndex(k1, k2)] += g[k1, k2]
    return vjp


# print UnvectorizeMatrix(vec)
UnvectorizeMatrix.defvjp(UnvectorizeMatrix_vjp)

UnvectorizeMatrixGrad = jacobian(UnvectorizeMatrix)
print UnvectorizeMatrixGrad(vec)

